# chapter 9 : 数据聚合与分组计算

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt

In [2]:
df = DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                'key2' : ['one', 'two', 'one', 'two', 'one'],
                'data1' : np.random.randn(5),
                'data2' : np.random.randn(5)})

In [3]:
grouped = df.groupby(df['key1'])
print(grouped.mean())

grouped2 = df['data1'].groupby(df['key2'])
print(grouped2.median())

         data1     data2
key1                    
a    -0.054371  0.759994
b     0.597138  0.747285
key2
one    0.136585
two    0.301165
Name: data1, dtype: float64


In [4]:
df['data1'].groupby([df['key1'], df['key2']]).mean()

key1  key2
a     one     0.146124
      two    -0.455361
b     one     0.136585
      two     1.057691
Name: data1, dtype: float64

In [5]:
df.groupby('key1').mean() # so, there is no need for df['key1']

,data1,data2
key1,,
a,-0.054371,0.759994
b,0.597138,0.747285


In [6]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

In [7]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  0.119554  0.946296
1    a  two -0.455361  1.434520
4    a  one  0.172694 -0.100834
b
  key1 key2     data1     data2
2    b  one  0.136585  0.088729
3    b  two  1.057691  1.405842


In [8]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print(k1, k2)
    print(group)

a one
  key1 key2     data1     data2
0    a  one  0.119554  0.946296
4    a  one  0.172694 -0.100834
a two
  key1 key2     data1    data2
1    a  two -0.455361  1.43452
b one
  key1 key2     data1     data2
2    b  one  0.136585  0.088729
b two
  key1 key2     data1     data2
3    b  two  1.057691  1.405842


In [9]:
pieces = dict(list(df.groupby('key1')))
print(pieces)

{'a':   key1 key2     data1     data2
0    a  one  0.119554  0.946296
1    a  two -0.455361  1.434520
4    a  one  0.172694 -0.100834, 'b':   key1 key2     data1     data2
2    b  one  0.136585  0.088729
3    b  two  1.057691  1.405842}


In [10]:
grouped_col = df.groupby(df.dtypes, axis=1)
dict(list(grouped_col))

{dtype('float64'):       data1     data2
 0  0.119554  0.946296
 1 -0.455361  1.434520
 2  0.136585  0.088729
 3  1.057691  1.405842
 4  0.172694 -0.100834, dtype('O'):   key1 key2
 0    a  one
 1    a  two
 2    b  one
 3    b  two
 4    a  one}

In [11]:
print(df['data1'].groupby(df['key1']).mean())
print(df[['data1']].groupby(df['key1']).mean())
print('\n')
print(type(df['data1']))
print(type(df[['data1']]))

key1
a   -0.054371
b    0.597138
Name: data1, dtype: float64
         data1
key1          
a    -0.054371
b     0.597138


<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>


In [12]:
people = DataFrame(np.random.randn(5, 5),
                   columns=['a', 'b', 'c', 'd', 'e'],
                   index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

groupby_map = {'a' : 'red', 'b' : 'blue', 'c' : 'red', 'd' : 'red', 'e' : 'blue'}
by_column = people.groupby(groupby_map, axis=1)

In [13]:
print(by_column.mean())
print(by_column.count())

            blue       red
Joe    -0.324997  0.314316
Steve   0.322023  0.072462
Wes     0.294400 -0.027082
Jim    -0.053160 -0.308520
Travis  0.876528 -0.234197
        blue  red
Joe        2    3
Steve      2    3
Wes        2    3
Jim        2    3
Travis     2    3


In [14]:
sl = ['sam', 'jack', 'luc', 'rat']
len(sl[0])

people.groupby(len).sum()

,a,b,c,d,e
3,0.634239,1.543097,-1.343155,0.645055,-1.710611
5,0.130566,0.093370,-0.218043,0.304864,0.550676
6,-0.533721,0.409010,-0.617939,0.449070,1.344046


In [15]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.881862  0.747811 -0.532137  0.286004 -1.397804
  two  1.355265  0.025639 -1.307125 -0.973700 -0.131958
5 one  0.130566  0.093370 -0.218043  0.304864  0.550676
6 two -0.533721  0.409010 -0.617939  0.449070  1.344046

In [16]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                     [1, 3, 5, 1, 3]], names=['cty', 'tenor'])
hier_df = DataFrame(np.random.randn(4, 5), columns=columns)

hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


In [17]:
hier_df.groupby(level='cty', axis=1).size()

cty
JP    2
US    3
dtype: int64

In [18]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
    
df.groupby('key1').agg(peak_to_peak)

,data1,data2
key1,,
a,0.628055,1.535354
b,0.921106,1.317113


In [19]:
df.groupby('key1').describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.054371  0.348282 -0.455361 -0.167904  0.119554  0.146124   
b      2.0  0.597138  0.651320  0.136585  0.366862  0.597138  0.827414   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.172694   3.0  0.759994  0.784448 -0.100834  0.422731  0.946296   
b     1.057691   2.0  0.747285  0.931340  0.088729  0.418007  0.747285   

                          
           75%       max  
key1                      
a     1.190408  1.434520  
b     1.076563  1.405842

In [20]:
grouped_summary = df.groupby('key1').describe()
grouped_summary['data1']['count']

key1
a    3.0
b    2.0
Name: count, dtype: float64

In [21]:
tips = pd.read_csv('tips.csv', header=0)
print(tips.columns)

Index(['total_bill', 'tip', 'smoker', 'day', 'time', 'size'], dtype='object')


In [22]:
grouped = tips.groupby(['day', 'smoker'])
grouped_pct = grouped['tip']
print(grouped_pct.agg('mean'))
print(grouped_pct.agg(peak_to_peak))

day   smoker
Fri   No        2.812500
      Yes       2.714000
Sat   No        3.102889
      Yes       2.875476
Sun   No        3.167895
      Yes       3.516842
Thur  No        2.673778
      Yes       3.030000
Name: tip, dtype: float64
day   smoker
Fri   No        2.00
      Yes       3.73
Sat   No        8.00
      Yes       9.00
Sun   No        4.99
      Yes       5.00
Thur  No        5.45
      Yes       3.00
Name: tip, dtype: float64


In [23]:
grouped_pct.agg(['mean', peak_to_peak])

mean  peak_to_peak
day  smoker                        
Fri  No      2.812500          2.00
     Yes     2.714000          3.73
Sat  No      3.102889          8.00
     Yes     2.875476          9.00
Sun  No      3.167895          4.99
     Yes     3.516842          5.00
Thur No      2.673778          5.45
     Yes     3.030000          3.00

In [24]:
grouped_pct.agg([('foo', np.mean), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      2.812500  0.898494
     Yes     2.714000  1.077668
Sat  No      3.102889  1.642088
     Yes     2.875476  1.630580
Sun  No      3.167895  1.224785
     Yes     3.516842  1.261151
Thur No      2.673778  1.282964
     Yes     3.030000  1.113491

In [25]:
np.mean([1, 2, 3, 4])

2.5

In [26]:
grouped.agg({'tip' : np.max, 'size' : 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [27]:
grouped.agg({'tip' : ['min', 'max', 'mean', 'std'], 'size' : 'sum'})

tip                            size
              min    max      mean       std  sum
day  smoker                                      
Fri  No      1.50   3.50  2.812500  0.898494    9
     Yes     1.00   4.73  2.714000  1.077668   31
Sat  No      1.00   9.00  3.102889  1.642088  115
     Yes     1.00  10.00  2.875476  1.630580  104
Sun  No      1.01   6.00  3.167895  1.224785  167
     Yes     1.50   6.50  3.516842  1.261151   49
Thur No      1.25   6.70  2.673778  1.282964  112
     Yes     2.00   5.00  3.030000  1.113491   40

In [28]:
tips.groupby(['day', 'smoker'], as_index=True).mean()

total_bill       tip      size
day  smoker                                
Fri  No       18.420000  2.812500  2.250000
     Yes      16.813333  2.714000  2.066667
Sat  No       19.661778  3.102889  2.555556
     Yes      21.276667  2.875476  2.476190
Sun  No       20.506667  3.167895  2.929825
     Yes      24.120000  3.516842  2.578947
Thur No       17.113111  2.673778  2.488889
     Yes      19.190588  3.030000  2.352941

In [29]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size
0,Fri,No,18.420000,2.812500,2.250000
1,Fri,Yes,16.813333,2.714000,2.066667
2,Sat,No,19.661778,3.102889,2.555556
3,Sat,Yes,21.276667,2.875476,2.476190
4,Sun,No,20.506667,3.167895,2.929825
5,Sun,Yes,24.120000,3.516842,2.578947
6,Thur,No,17.113111,2.673778,2.488889
7,Thur,Yes,19.190588,3.030000,2.352941


In [30]:
k1_means = df.groupby('key1').mean()
print(k1_means)

         data1     data2
key1                    
a    -0.054371  0.759994
b     0.597138  0.747285


In [31]:
k1_means.add_prefix('mean_')

,mean_data1,mean_data2
key1,,
a,-0.054371,0.759994
b,0.597138,0.747285


In [32]:
people.groupby(['one', 'two', 'one', 'two', 'one']).transform(np.mean)

,a,b,c,d,e
Joe,-0.418249,0.642156,-0.217990,0.689275,-0.078202
Steve,0.742915,0.059504,-0.762584,-0.334418,0.209359
Wes,-0.418249,0.642156,-0.217990,0.689275,-0.078202
Jim,0.742915,0.059504,-0.762584,-0.334418,0.209359
Travis,-0.418249,0.642156,-0.217990,0.689275,-0.078202


In [33]:
def demean(arr):
    return arr - arr.mean()

demeaned = people.groupby(['one', 'two', 'one', 'two', 'one']).transform(demean)
print(demeaned)

               a         b         c         d         e
Joe     0.579085  0.105654  0.714097 -0.403271 -1.319602
Steve  -0.612350  0.033865  0.544541  0.639282  0.341317
Wes    -0.463613  0.127492 -0.314147  0.643477 -0.102646
Jim     0.612350 -0.033865 -0.544541 -0.639282 -0.341317
Travis -0.115472 -0.233146 -0.399950 -0.240205  1.422248


In [34]:
frame = DataFrame({'data1' : np.random.randn(1000),
                   'data2' : np.random.randn(1000)})

factor = pd.cut(frame.data1, 4)

def get_stats(group):
    return {'min' : group.min(), 'max' : group.max(), 'count' : group.count(), 'mean' : group.mean()}

grouped = frame.data2.groupby(factor)

grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.143, -1.429]",-2.432713,2.078720,80.0,0.027646
"(-1.429, 0.277]",-3.198654,3.044196,533.0,-0.008790
"(0.277, 1.984]",-2.508823,3.097926,356.0,0.015627
"(1.984, 3.69]",-1.746595,2.258407,31.0,0.056461


In [39]:
fec = pd.read_csv('P00000001-ALL.csv')
# fec.head(5)
print(fec.shape)
print(fec.iloc[1000])

(1001731, 16)
cmte_id                          C00431171
cand_id                          P80003353
cand_nm                       Romney, Mitt
contbr_nm            NEUWIEN, SUSAN W. MS.
contbr_city                     ENTERPRISE
contbr_st                               AL
contbr_zip                     3.63302e+08
contbr_employer                    RETIRED
contbr_occupation                  RETIRED
contb_receipt_amt                     1000
contb_receipt_dt                 13-FEB-12
receipt_desc                           NaN
memo_cd                                NaN
memo_text                              NaN
form_tp                              SA17A
file_num                            780124
Name: 1000, dtype: object


In [42]:
unique_cands = fec.cand_nm.unique()
print(unique_cands)

['Bachmann, Michelle' 'Romney, Mitt' 'Obama, Barack'
 "Roemer, Charles E. 'Buddy' III" 'Pawlenty, Timothy' 'Johnson, Gary Earl'
 'Paul, Ron' 'Santorum, Rick' 'Cain, Herman' 'Gingrich, Newt'
 'McCotter, Thaddeus G' 'Huntsman, Jon' 'Perry, Rick']
